# 🌊 Flash Flood Detection in Arizona Using VideoDB RTStream

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/real_time_streaming/Flash_Flood_Detection.ipynb)

## 📖 Storytime: Why This Matters

The stunning Arizona deserts, known for their dry riverbeds and scenic beauty, hide a deadly risk.  
During the summer monsoon, sudden torrential rains can trigger **flash floods** in these seemingly harmless dry zones — with little or no warning.

Conventional alert systems relying on rain gauges or weather satellites often fail to deliver timely, location-specific warnings. By the time a danger alert is sent, it might already be too late.

**But we have a smarter way.**

With **VideoDB RTStream**, we can install real-time cameras near flood-prone areas and let AI continuously monitor the visuals.  
As soon as the AI detects signs of a flash flood — like a sudden surge of water through dry land — it can instantly send alerts, giving local authorities and tourists precious moments to act.

---

## 🚀 What You’ll Build in This Notebook

In this notebook, we’ll create a real-time flash flood detection system using **VideoDB RTStream**.  
By the end of this demo, you’ll learn how to:
- Connect a live video stream to VideoDB
- Use AI to continuously analyze scenes for signs of a flash flood
- Detect a **flash flood event**
- Trigger a real-time alert when detected

Let’s build it together!



---

## 📦 Step 1: Install Dependencies

Before setting up our AI-powered flood monitor, let’s install the required VideoDB SDK.

In [1]:
!pip install -q videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Let's connect to VideoDB's API using your credentials to prepare for stream monitoring.

Please enter your `VIDEO_DB_API_KEY` in the input box that appears below after you run this cell.

Your input will be masked.


In [2]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---

## 📦 Step 3: Connect to the Arizona Flood RTSP Stream

Connect to the live camera stream monitoring a flood-prone desert area.

In this demo, the stream is running at `rtsp://samples.rts.videodb.io:8554/floods`.

In [ ]:
rtsp_url = "rtsp://samples.rts.videodb.io:8554/floods"
flood_stream = coll.connect_rtstream(
    name="Arizona Flood Stream",
    url=rtsp_url,
)
print(flood_stream)


RTStream(id=rts-019719b2-0c84-7a71-a037-311855a8d160, name=Arizona Flood Stream, collection_id=None, created_at=None, sample_rate=30, status=connected)


#### Let us list all the rtstreams in our collection.

In [ ]:
def list_rtstreams():
    for rtstream in coll.list_rtstreams():
        print(f"""RTStream:
        ID            : {rtstream.id}
        Name          : {rtstream.name}
        Collection ID : {rtstream.collection_id}
        Created At    : {rtstream.created_at}
        Sample Rate   : {rtstream.sample_rate}
        Status        : {rtstream.status}
        """)
        print("-" * 80)

list_rtstreams()

RTStream:
        ID            : rts-019719b2-0c84-7a71-a037-311855a8d160
        Name          : Arizona Flood Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-29T01:39:48.484134
        Sample Rate   : 30
        Status        : connected
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019711db-1086-7750-ba79-8f47a4fed603
        Name          : Cricket Finals Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T13:07:38.758741
        Sample Rate   : 30
        Status        : stopped
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019711a0-0fde-7911-b282-25bc0b4ecf65
        Name          : Baby Crib Monitor
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T12:03:11.966358



#### If you have already connected the stream, run the below cell with the **rtstream id** to reconnect.

In [3]:
# flood_stream = coll.get_rtstream("")

In [ ]:
# To stop the stream
# flood_stream.stop()

In [4]:
# To start the stream
# flood_stream.start()

---
### 👀 Let's have a look at the riverbed


#### 📺 Helper Function: Display Video Stream

This cell contains a small utility function to help visualize the video streams with helpful information. You don't need to modify this code.

In [5]:
# To display the stream with relevant information

from IPython.display import HTML
import re
from datetime import datetime
from videodb import play_stream

def display_stream(video_url, video_name="🎥 Camera Feed"):

    match = re.search(r'/(\d{16})-(\d{16})\.m3u8', video_url)
    if match:
        start_ts = int(match.group(1)) / 1e6
        end_ts = int(match.group(2)) / 1e6
        start_time = datetime.utcfromtimestamp(start_ts).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.utcfromtimestamp(end_ts).strftime('%Y-%m-%d %H:%M:%S')
        time_range = f"{start_time} → {end_time} UTC"
    else:
        time_range = "Time Unknown"

    video_player_html = play_stream(video_url)

    return HTML(f"""
    <div style="position:relative;width:640px;">
      {video_player_html._repr_html_() if hasattr(video_player_html, '_repr_html_') else video_player_html}
      <div style="position:absolute;top:10px;left:10px;background:rgba(0,0,0,0.6);color:#fff;padding:6px 12px;border-radius:4px;font-size:13px;font-family:sans-serif;">
        <strong>{video_name}</strong><br>{time_range}
      </div>
    </div>
    """)


#### 🔗 Get & Display Recent Stream

This cell uses the helper function above to fetch and display the last few minutes of the stream.

In [8]:
# To get last few minutes stream link
import time

def fetch_stream(rtstream):

    now = int(time.time())
    start = int(now - (5 * 60))
    stream_url = rtstream.generate_stream(start, now)
    return stream_url

video_url = fetch_stream(flood_stream)

video_name = "🌊 Arizona Desert · Flash Flood Detection"
display_stream(video_url , video_name)


---

## 📦 Step 4: Index Scenes and Detect Flash Floods

We’ll create a real-time scene index that periodically analyzes video frames and generates natural language descriptions of what’s happening in the stream.

The AI will look for sudden visual cues of water flooding dry land and describe them.


In [ ]:
from videodb import SceneExtractionType

flood_scene_index = flood_stream.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={
        "time": 5,
        "frame_count": 3,
    },
    prompt="Monitor the dry riverbed and surrounding area. If moving water is detected across the land, identify it as a flash flood and describe the scene.",
    name="Flash_Flood_Detection_Index"
)
flood_index_id = flood_scene_index.rtstream_index_id
print("Scene Index ID:", flood_index_id)


Scene Index ID: 318f52d1a4524559


#### Let us list the scene indexes created on our rtstream.

In [ ]:
def list_rtstream_indexes(rtstream):
    # List live stream indexes
    rtstream_indexes = rtstream.list_scene_indexes()
    for rtstream_index in rtstream_indexes:

        print(f"""RTStreamSceneIndex:
            Index ID       : {rtstream_index.rtstream_index_id}
            RTStream ID    : {rtstream_index.rtstream_id}
            Name           : {rtstream_index.name}
            Status         : {rtstream_index.status}
            Config         : {rtstream_index.extraction_config}
            Prompt         : {rtstream_index.prompt}
        """)
        print("-" * 80)

list_rtstream_indexes(flood_stream)

RTStreamSceneIndex:
            Index ID       : 318f52d1a4524559
            RTStream ID    : rts-019719b2-0c84-7a71-a037-311855a8d160
            Name           : Flash_Flood_Detection_Index
            Status         : running
            Config         : {'frame_count': '3', 'time': '5'}
            Prompt         : Monitor the dry riverbed and surrounding area. If moving water is detected across the land, identify it as a flash flood and describe the scene.
        
--------------------------------------------------------------------------------



#### If you have already created a scene index, run the below cell with your **scene index id** to reconnect.

In [ ]:
# flood_index_id = ""
# flood_scene_index = flood_stream.get_scene_index(flood_index_id)

In [ ]:
# To stop the index
# flood_scene_index.stop()

In [ ]:
# To start the index
# flood_scene_index.start()

---
### Let us see the result of the scene indexing

In [ ]:
import time
from datetime import datetime
from zoneinfo import ZoneInfo

def _convert_to_ist(timestamp: float) -> str:
    """Convert UTC timestamp to IST (Asia/Kolkata) datetime string."""
    return (
        datetime.fromtimestamp(timestamp)
        .astimezone(ZoneInfo("Asia/Kolkata"))
        .strftime("%Y-%m-%d %H:%M:%S")
    )

def get_scenes(rtstream, index_id):
    # Print indexed scenes
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    scenes = rtstream_scene_index.get_scenes(page_size=5)
    # print(scenes["scenes"][:2])
    if scenes:
        for scene in scenes.get("scenes"):
            start = _convert_to_ist(scene["start"])
            end = _convert_to_ist(scene["end"])
            description = scene["description"]
            print(f"{start}-{end}: {description}")
            print("-" * 80)
    else:
        print("Scenes not found for given index.")

get_scenes(flood_stream , flood_index_id)

2025-05-29 07:17:51-2025-05-29 07:17:56: **Flash Flood Detected**

The images show a flash flood occurring in a dry riverbed. The water is flowing rapidly and forcefully through the narrow channels carved into the red rock landscape. The water is churning and appears muddy, indicating it is carrying sediment and debris. The surrounding area is arid, with sparse vegetation. People are observing the flash flood from the rocky banks. The sky is overcast, suggesting recent rainfall may have contributed to the flash flood.
--------------------------------------------------------------------------------
2025-05-29 07:17:46-2025-05-29 07:17:50: **Flash Flood Detected**

The scene depicts a flash flood surging through a narrow, rocky riverbed. The water is turbulent and fast-moving, churning with white foam as it rushes between the reddish-brown rock formations. The water level is high, filling the channel completely and indicating a sudden and significant increase in water volume. The surroun

---

## 📦 Step 5: Define a Flash Flood Event

Now, we’ll define an event type in the system to detect visual signs of a flash flood.


In [ ]:
flood_event_id = conn.create_event(
    event_prompt="Detect sudden flash floods or water surges.",
    label="flash_flood"
)
print("Event ID:", flood_event_id)


Event ID: 7929a3ecc4624dbb


---

## 📦 Step 6: Attach an Alert to the Flash Flood Event

Finally, we’ll link a real-time alert to this event, which will send a notification to our webhook the moment a flash flood is detected.


In [ ]:
# Enter link to your webhook url where you want alerts to go. You can create one simply on pipedream.
webhook_url = ""

if webhook_url:
  flood_alert_id = flood_scene_index.create_alert(
      flood_event_id,
      callback_url=webhook_url
  )
  print("Alert ID:", flood_alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

Alert ID: d9c6038e631071cd


---

## 📡 Example Alert Payload

When a flash flood occurs, the system sent this alert to the webhook url:

```json
{
  "event_id": "event-7929a3ecc4624dbb",
  "label": "flash_flood",
  "confidence": 0.95,
  "explanation": "The scene analysis clearly depicts a flash flood with rapidly moving water, debris, and a recent heavy rainfall event, aligning strongly with the alert context of detecting sudden flash floods or water surges.",
  "timestamp": "2025-05-29T01:51:33.907778+00:00",
  "start_time": "2025-05-29T07:20:34.289908+05:30",
  "end_time": "2025-05-29T07:20:39.891267+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019719b2-0c84-7a71-a037-311855a8d160/1748483434000000-1748483440000000.m3u8"
}
```

In [6]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-019719b2-0c84-7a71-a037-311855a8d160/1748483434000000-1748483440000000.m3u8"
video_name = "🌊 Arizona Desert · flash_flood"

display_stream(alert_stream_url, video_name)

#### Let us stop the flood index and proceed with the notebook to explore more possibilities.

In [ ]:
# flood_scene_index.stop()

---
### Let us set up some other alerts that are necessary
1. Heavy rainfall, detecting heavy rainfall early can help us predict a flash flood occurence
2. Detect the presence of a person stuck in the flash flood, for immediate rescue  

We can start with creating a new scene index for monitoring rainfall or a person stuck in the place.

In [ ]:
riverbed_monitoring_scene_index = flood_stream.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={
        "time": 15,
        "frame_count": 1,
    },
    prompt="Monitor the dry riverbed and surrounding area. In case you detect heavy rainfall mention 'heavy rainfall detected'. If you detect a person stuck in the area during rainfall or flash flood mention 'person detected, rescue needed'",
    name="Riverbed_Monitoring_Index"
)
riverbed_monitoring_index_id = riverbed_monitoring_scene_index.rtstream_index_id
print("Scene Index ID:", riverbed_monitoring_index_id)


Scene Index ID: a64531c43284cc99


#### Checking the list of indexes

In [ ]:
list_rtstream_indexes(flood_stream)

RTStreamSceneIndex:
            Index ID       : 318f52d1a4524559
            RTStream ID    : rts-019719b2-0c84-7a71-a037-311855a8d160
            Name           : Flash_Flood_Detection_Index
            Status         : stopped
            Config         : {'frame_count': '3', 'time': '5'}
            Prompt         : Monitor the dry riverbed and surrounding area. If moving water is detected across the land, identify it as a flash flood and describe the scene.
        
--------------------------------------------------------------------------------
RTStreamSceneIndex:
            Index ID       : a64531c43284cc99
            RTStream ID    : rts-019719b2-0c84-7a71-a037-311855a8d160
            Name           : Riverbed_Monitoring_Index
            Status         : running
            Config         : {'frame_count': '1', 'time': '15'}
            Prompt         : Monitor the dry riverbed and surrounding area. In case you detect heavy rainfall mention 'heavy rainfall detected'. If you

---
### Let's have a look at generated scenes

In [ ]:
get_scenes(flood_stream , riverbed_monitoring_index_id)

2025-05-29 07:34:50-2025-05-29 07:35:05: heavy rainfall detected
--------------------------------------------------------------------------------
2025-05-29 07:34:34-2025-05-29 07:34:49: No heavy rainfall detected. No person detected.
--------------------------------------------------------------------------------
2025-05-29 07:34:19-2025-05-29 07:34:34: No heavy rainfall detected. No person detected.
--------------------------------------------------------------------------------
2025-05-29 07:34:03-2025-05-29 07:34:18: 'person detected, rescue needed'
--------------------------------------------------------------------------------
2025-05-29 07:33:47-2025-05-29 07:34:02: heavy rainfall detected
--------------------------------------------------------------------------------


---
### Now we can setup events and alerts for the index

1. Rainfall Detection

In [ ]:
# Create rainfall event
rainfall_event_id = conn.create_event(
    event_prompt="Detect heavy rainfall.",
    label="heavy_rainfall"
)
print("Event ID:", rainfall_event_id)

Event ID: df1611d13772e09b


In [ ]:
# Create rainfall alert

# Enter link to your webhook url where you want alerts to go.
rainfall_webhook_url = ""

if rainfall_webhook_url:
  rainfall_alert_id = riverbed_monitoring_scene_index.create_alert(
      rainfall_event_id,
      callback_url=rainfall_webhook_url
  )
  print("Alert ID:", rainfall_alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

Alert ID: ae31ea7c92a61bc7


---
2. Human detection for rescue

In [ ]:
# Create rescue event
rescue_event_id = conn.create_event(
    event_prompt="Detect if there is a person",
    label="human_rescue"
)
print("Event ID:", rescue_event_id)

Event ID: 717a1bcf7a705ec7


In [ ]:
# Create rescue alert

# Enter link to your webhook url where you want alerts to go.
rescue_webhook_url = ""

rescue_alert_id = riverbed_monitoring_scene_index.create_alert(
    rescue_event_id,
    callback_url=rescue_webhook_url
)
print("Alert ID:", rescue_alert_id)

Alert ID: 766872338353f95b


---
### Let us see the list of alerts associated with the `riverbed_monitoring_scene_index`

In [ ]:
def list_rtstream_alerts(rtstream, index_id):
    """
    Prints a list of alerts associated with a given scene index.
    """
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    alerts = rtstream_scene_index.list_alerts()

    for alert in alerts:
        print(f"""🔔 RTStream Alert:
    Alert ID      : {alert['alert_id']}
    Event ID      : {alert['event_id']}
    Label         : {alert['label']}
    Prompt        : {alert['prompt']}
    Status        : {alert['status']}
        """)
        print("-" * 80)

list_rtstream_alerts(flood_stream, riverbed_monitoring_index_id)


🔔 RTStream Alert:
    Alert ID      : 766872338353f95b
    Event ID      : 717a1bcf7a705ec7
    Label         : human_rescue
    Prompt        : Detect if there is a person
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : ae31ea7c92a61bc7
    Event ID      : df1611d13772e09b
    Label         : heavy_rainfall
    Prompt        : Detect heavy rainfall.
    Status        : enabled
        
--------------------------------------------------------------------------------


---

## 📡 Example Alert Payload

When the AI detected people near the flooding riverbed, the system sent this alert to the webhook url:

```json
{
  "event_id": "event-717a1bcf7a705ec7",
  "label": "human_rescue",
  "confidence": 0.95,
  "explanation": "Flash flood is detected and a person is present, indicating a potential need for rescue. High confidence due to explicit rescue request.",
  "timestamp": "2025-05-29T02:19:56.724411+00:00",
  "start_time": "2025-05-29T07:49:49.789911+05:30",
  "end_time": "2025-05-29T07:50:04.789911+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019719b2-0c84-7a71-a037-311855a8d160/1748485189000000-1748485205000000.m3u8"
}
```

In [7]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-019719b2-0c84-7a71-a037-311855a8d160/1748485189000000-1748485205000000.m3u8"
video_name = "🌊 Arizona Desert · human_rescue"

display_stream(alert_stream_url, video_name)

---

## 🌙 Wrapping Up: Real-Time Environmental Safety

With this system in place, communities, tourists, and local authorities in Arizona’s desert regions can receive **immediate alerts** when a dangerous flash flood occurs — gaining critical seconds to take cover, clear routes, or initiate rescues.

---

## 🔥 What Else Could We Monitor?

This system isn’t limited to flash floods. The same AI-driven video monitoring approach can protect lives in other natural disasters too.

---

### 🌲 Forest Fire Detection

**Why it’s needed:**  
In remote forest areas, smoke plumes and early fire flickers often go unnoticed for several minutes before traditional sensors or satellites pick them up.

**How we can do it better:**  
With AI-powered cameras watching key zones, we can detect rapid smoke build-ups or visible flames long before automated sensors trigger.

**Example indexing prompt:**

```markdown
"Monitor the forest area carefully. Detect sudden rising smoke, increasing haze, or visible flames. Clearly describe if a fire outbreak is visibly starting."
```

---

### 🌊 Sudden Tsunami Detection

**Why it’s needed:**  
Even with tsunami sensors and ocean buoys, near-shoreline towns often get only minutes of warning. Visual cues like water rapidly pulling away from shore or unusually large approaching waves are immediate, reliable signs.

**How we can do it better:**  
Install AI cameras on popular beaches and coastlines to detect rapid water retreat or walls of water approaching.

**Example indexing prompt:**

```markdown
"Watch the beach and shoreline closely. Detect unusual rapid water withdrawal from the shore, or approaching large waves indicating a possible tsunami. Describe the situation clearly."
```

---

### 🏔️ Landslide / Avalanche Detection

**Why it’s needed:**  
Mountain highways and tourist trails are often cut off by sudden landslides or snow avalanches, where every second matters for evacuation and road closures.

**How we can do it better:**  
Place AI cameras at landslide-prone mountain slopes or snow-covered passes to detect falling rocks, moving soil, or snow slides in real time.

**Example indexing prompt:**

```markdown
"Monitor the mountain slope area carefully. Detect falling rocks, soil displacement, or sudden snow slides. Clearly describe if a landslide or avalanche is starting."
```

---

## 🌍 The Possibilities Are Endless

Real-time AI video monitoring isn’t just for cities and homes — it can actively save lives in wild, unpredictable environments too.

**What natural threat would *you* monitor next?**